In [6]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9, 8]
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import glob
import os

In [7]:
def imshow(image):
    if (len(image.shape)==2):
        plt.imshow(image,cmap="gray")
    else:
        plt.imshow(image[:,:,::-1])
def resize(img, ratio):
    width = int(img.shape[1] * ratio)
    height = int(img.shape[0] * ratio)
    dim = (width, height)
    return cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


In [8]:
def iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xTopLeft = max(boxA[0], boxB[0])
    yTopLeft = max(boxA[1], boxB[1])
    xBottomRight = min(boxA[2], boxB[2])
    yBottomRight = min(boxA[3], boxB[3])
    # caculate intersection rectangle's area
    interArea = max(0, xBottomRight - xTopLeft + 1) * max(0, yBottomRight - yTopLeft + 1)
    # caculate boxes's area
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # union area = areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [9]:
# get coordinate of correct bounding box
def getCorrectBound(filename):
    name = filename.rsplit('.', 1)[0]
    tree = ET.parse(name + ".xml")
    root = tree.getroot();
    bounds = root.findall('.//bndbox')

    # variable for store bounding box
    bound_coordinates = []
    for bound in bounds:
        # xmin, ymin, xmax, ymax
        coordinate = []
        for tag in bound.findall('./*'):
            coordinate.append(int(tag.text));
        bound_coordinates.append(coordinate)

    #draw bounding box on picture
    image = cv2.imread(filename)
    copy_image = image.copy()

    for bound in bound_coordinates:
        cv2.rectangle(copy_image,(bound[0],bound[1]),(bound[2],bound[3]),(0,0,255),1)
    return bound_coordinates

In [10]:
path = 'train/'
iou_arr = []
total_correct_cells = 0
precision_arr = []

for filename in glob.glob(os.path.join(path, '*.png')):
    image = cv2.imread(filename)
    result = image.copy()
    result = resize(result, 4)
    
    correctCellsImage = 0
    cellsImage = 0
    
    # correct bound
    bound_coordinates = getCorrectBound(filename)
    
    # detect edge
    gray = cv2.cvtColor(result,cv2.COLOR_BGR2GRAY)   
    canny = cv2.Canny(gray, 10, 50)
    cannyres = canny.copy()

    #remove_horizontal
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
    remove_horizontal = cv2.morphologyEx(cannyres, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
    cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(canny, [c], -1, (0,0,0), 4)

    #remove_vertical
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,28))
    remove_vertical = cv2.morphologyEx(cannyres, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(canny, [c], -1, (0,0,0), 4)
    
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (12, 6))
    connected = cv2.dilate(canny, kernel2, iterations=2)
    connected = cv2.erode(connected, kernel2, iterations=2)

    n_connected = resize(connected, 1/4)
    contours, hierarchy = cv2.findContours(n_connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    dt_bounds = []
    image1 = image.copy()
    for idx in range(len(contours)):
        x, y, w, h = cv2.boundingRect(contours[idx])
        if w > 4 and h > 4:
            cv2.rectangle(image1, (x, y), (x+w-1, y+h-1), (255, 0, 0), 1)
            dt_bound = [x,y,x+w-1,y+h-1]
            dt_bounds.append(dt_bound)
    for bound in bound_coordinates:
        max_iou = 0
        for dt_bound in dt_bounds:
            iou_value = iou(dt_bound,bound)
            max_iou = max(max_iou,iou_value)
        if (max_iou > 0.5):
            total_correct_cells += 1
            correctCellsImage += 1
        cellsImage += 1
        iou_arr.append(max_iou)
    precisionImage = (filename,round(correctCellsImage/cellsImage,2))
    precision_arr.append(precisionImage)
average_iou = sum(iou_arr)/len(iou_arr)
print("Average of iou is %s" %average_iou)
print("Total of correct cells is %s" %total_correct_cells)
sum_pre = 0
for pre in precision_arr: 
    sum_pre += pre[1]
average_pre = sum_pre/len(precision_arr)
print("Average precision is %s" %average_pre)
for precision in precision_arr:
    print("%s %s" %(precision[0],precision[1]))

Average of iou is 0.7672603070039091
Total of correct cells is 3922
Average correct cells is 0.8655833333333329
train\19.png 1.0
train\20.png 0.96
train\32.png 0.93
train\33.png 0.95
train\34.png 0.55
train\48643.png 0.93
train\48649.png 1.0
train\48725.png 0.94
train\48726.png 1.0
train\48727.png 0.5
train\48811.png 0.72
train\48812.png 0.95
train\48851.png 0.79
train\48854.png 0.55
train\48855.png 1.0
train\48856.png 1.0
train\48865.png 1.0
train\48866.png 0.96
train\48867.png 0.95
train\48868.png 1.0
train\48968.png 0.97
train\48969.png 1.0
train\48971.png 0.86
train\48972.png 0.96
train\49.png 0.89
train\49009.png 0.83
train\49035.png 0.76
train\49038.png 0.8
train\49044.png 1.0
train\49045.png 1.0
train\49046.png 1.0
train\49047.png 0.89
train\49080.png 1.0
train\49162.png 0.8
train\49170.png 0.83
train\49171.png 0.85
train\49174.png 1.0
train\49176.png 0.82
train\49188.png 0.97
train\49210.png 0.96
train\49211.png 0.92
train\49216.png 0.96
train\49217.png 1.0
train\49218.png 0.94